## MLflow's Model Registry

In [5]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

#### Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI

In [7]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='ths-skin-cancer-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-cool-exp', tags={}>]

In [9]:
exp_name = "my-cool-exp"
#client.create_experiment(exp_name)

In [10]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.val_acc > .25",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results = 3,
order_by = ['metrics.val_acc ASC']
)


In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['val_acc']:.4f}")

run id: a60df699fffe43b7b4d0f2b417eb7ed5, rmse: 0.2667
run id: 6bbb818e9e5246d7a2ac7a016cd7e690, rmse: 0.3303
run id: 04043070e0d740a4b4f4889b6147ff5a, rmse: 0.3303


#### Interacting with model registry
creat new verion of "skin-cancer-classifier"

In [12]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


run_id = "04043070e0d740a4b4f4889b6147ff5a"
model_name = "skin-cancer-classifier"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name=model_name)


Registered model 'skin-cancer-classifier' already exists. Creating a new version of this model...
2022/09/05 13:11:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: skin-cancer-classifier, version 7
Created version '7' of model 'skin-cancer-classifier'.


<ModelVersion: creation_timestamp=1662360060485, current_stage='None', description=None, last_updated_timestamp=1662360060485, name='skin-cancer-classifier', run_id='04043070e0d740a4b4f4889b6147ff5a', run_link=None, source='./mlruns/1/04043070e0d740a4b4f4889b6147ff5a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [13]:
## check model version
latest_verions = client.get_latest_versions(name=model_name)


for version in latest_verions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Archived
version: 3, stage: Production
version: 6, stage: Staging
version: 7, stage: None


In [14]:
## set model to Staging
model_version = 6
new_stage = "Staging"
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1662355041620, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2022-09-05', last_updated_timestamp=1662360063768, name='skin-cancer-classifier', run_id='04043070e0d740a4b4f4889b6147ff5a', run_link=None, source='./mlruns/1/04043070e0d740a4b4f4889b6147ff5a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [15]:
from datetime import datetime
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1662355041620, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2022-09-05', last_updated_timestamp=1662360064557, name='skin-cancer-classifier', run_id='04043070e0d740a4b4f4889b6147ff5a', run_link=None, source='./mlruns/1/04043070e0d740a4b4f4889b6147ff5a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

### Compare models base on stages

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root_path = r"E:\deep_learning\skin_cancer\dataset\dataset"
img_width,img_height = 128,128
model_input = (img_width,img_height,3)
img_size = (img_width,img_height)
batch_size = 32

_datagen = ImageDataGenerator(
        rescale=1/255,
		width_shift_range=0.1,
		height_shift_range=0.1,
		horizontal_flip=True)

test_gen = _datagen.flow_from_directory(
    root_path+"/test",
    target_size = img_size,
    batch_size=batch_size,
    class_mode = "categorical"
)

Found 330 images belonging to 3 classes.


In [23]:
from datetime import datetime
def test_model(name, stage, test_gen,):
    model_path = f"models:/{name}/{stage}"
    tf_model = mlflow.keras.load_model(model_path)

    results = tf_model.evaluate(test_gen , batch_size=batch_size)

    return {"val_loss": results[0],"val_acc" : results[1]}

In [24]:
%time test_model(name=model_name, stage="Production",test_gen=test_gen)

11/11 [==============================] - 10s 385ms/step - loss: 1.0986 - accuracy: 0.3333
CPU times: total: 7.31 s
Wall time: 13.8 s


{'val_loss': 1.0985932350158691, 'val_acc': 0.3333333432674408}

In [27]:
%time test_model(name=model_name, stage="Staging",test_gen=test_gen)

11/11 [==============================] - 2s 127ms/step - loss: 1.0994 - accuracy: 0.2667
CPU times: total: 3.16 s
Wall time: 3.03 s


{'val_loss': 1.0993913412094116, 'val_acc': 0.2666666805744171}